# Rule Summarization using bedrock

# SETUP

In [1]:
!which python
import sys
sys.version

/usr/local/bin/python


'3.10.13 (main, Oct 12 2023, 10:20:20) [GCC 12.2.0]'

## Install required libraries

In [ ]:
!pip install --upgrade pip

In [2]:
!pip install ipywidgets --quiet

In [ ]:
!pip install boto3 #--quiet
!pip install langchain #--quiet

In [ ]:
!pip install torch # --quiet

In [ ]:
!pip install sentence_transformers # --quiet

In [ ]:
!pip install chromadb #--quiet

In [6]:
!pip install typing-extensions --upgrade

In [ ]:
!pip install SQLAlchemy==2.0.1 --quiet
!pip install flask-sqlalchemy --quiet


In [ ]:
!pip3 install -U unstructured

In [2]:
import unstructured 

In [ ]:
!pip install pdf2image
!pip install pdfminer
!pip install pdfminer.six

In [ ]:
!pip install rouge

In [ ]:
!pip3 install opencv-python-headless --quiet #required for cv2

## Import libraries

In [1]:
import chromadb 

In [2]:
import sentence_transformers

In [3]:
import langchain

In [4]:

from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredPDFLoader
#from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import NLTKTextSplitter, CharacterTextSplitter
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import os



from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.llms.bedrock import Bedrock


## Setup LLMs

### Setup the Claude LLM with LangChain 

In [5]:

model_kwargs = { 
    "max_tokens_to_sample": 1000,
    "temperature": 0.1,
    "top_p": 0.9,
    }
    
llm = Bedrock(
        credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"), #sets the profile name to use for AWS credentials (if not the default)
        region_name=os.environ.get("BWB_REGION_NAME"), #sets the region name (if not the default)
        endpoint_url=os.environ.get("BWB_ENDPOINT_URL"), #sets the endpoint URL (if necessary)
        model_id="anthropic.claude-v2", #set the foundation model
        model_kwargs=model_kwargs,
       # streaming=True,
       # callbacks=[StreamingStdOutCallbackHandler()]
        ) #configure the properties for Claude

In [ ]:
!pip install openai

### Setup local LLM

In [42]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
openai.api_key = "EMPTY" # Not support yeta
openai.api_base = "http://10.64.10.36:8000/v1"
import os
os.environ["OPENAI_API_KEY"] = "EMPTY"
os.environ["OPENAI_API_BASE"] = "http://10.64.10.36:8000/v1"

llm = OpenAI(model="")

In [6]:
llm("Human: Puede generar un sumario ficticion de una corte de argentina? Assistant:")

' Aquí está un sumario ficticio de una corte de Argentina:\n\nCorte Suprema de Justicia de la Nación Argentina\n\nCaso: Pérez vs. García\nN° de Expediente: 1234/2022 \n\nPartes:\n- Demandante: Juan Pérez\n- Demandado: Pedro García\n\nHechos: \nEl Sr. Juan Pérez presentó una demanda contra el Sr. Pedro García por daños y perjuicios. Pérez alega que García chocó su automóvil contra la propiedad del demandante, específicamente contra la pared del frente de su casa, causando daños materiales. Pérez solicita una indemnización por los daños ocasionados.\n\nGarcía niega haber chocado contra la propiedad del demandante. Alega que el día del supuesto incidente se encontraba en otra localidad. \n\nProcedimiento:\n- La demanda fue presentada ante el Juzgado Civil de Primera Instancia.\n- Se realizó una audiencia preliminar donde las partes presentaron sus argumentos. \n- Se fijó una fecha para la audiencia de juicio.\n- En la audiencia de juicio se presentaron testigos y pruebas documentales.\n- 

### Test the LLM

In [28]:
llm("\n\nHuman:Can you provide me the list of planets on the solar system?\n\nAssistant:")

' Here is a list of the planets in our solar system:\n\n- Mercury\n- Venus \n- Earth\n- Mars\n- Jupiter\n- Saturn \n- Uranus\n- Neptune\n\nThe order listed is their order from the Sun. Mercury is the closest planet to the Sun, while Neptune is the farthest.\n\nThere used to be 9 planets, but Pluto was reclassified as a dwarf planet in 2006, so the number of official planets dropped to 8.'

## Setup Embeddings

### Setup AWS Titan  for calculating embeddings

In [7]:
embeddings = BedrockEmbeddings(
        credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"), #sets the profile name to use for AWS credentials (if not the default)
        region_name=os.environ.get("BWB_REGION_NAME"), #sets the region name (if not the default)
        endpoint_url=os.environ.get("BWB_ENDPOINT_URL"),
)#sets the endpoint URL (if necessary)

### Setup HF embeddings

In [6]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [46]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

# INTERACTIVE EXAMPLE

## Load a PDF and calculate the embeddings

In [8]:
loader = UnstructuredPDFLoader("/home/harpo/JurisGPT/rawdata/laboral/10000003368.pdf")

In [9]:
data = loader.load()

In [17]:
data[0].page_content

'SUPREMA CORTE DE JUSTICIA - SALA SEGUNDA\n\nPODER JUDICIAL MENDOZA\n\nfoja: 15\n\nCUIJ: 13-05340332-2/2((010402-161169))\n\nPIEZA SEPARADA EN AUTOS N° 13-05340332-2/1 PROVINCIA ART S.A. EN J° ABACA ROXANA CARINA C/ PROVINCIA ART SA P/ ACCIDENTE (161169) P/ PLENARIO\n\n105982886*\n\nEn Mendoza, a 11 días del mes de octubre del año 2022, reunida la Suprema Corte de Justicia, tomó en consideración para dictar sentencia plenaria en la pieza separada CUIJ N° 13-05340332-2/2, caratulada: "PIEZA SEPARADA EN AUTOS N° 13-05340332-2/1 PROVINCIA ART S.A. EN J° ABACA ROXANA CARINA C/ PROVINCIA ART S.A. P/ ACCIDENTE P/ PLENARIO".\n\nConforme a lo decretado a fs. 14 se deja constancia del orden de estudio efectuado en la causa para el tratamiento de las cuestiones por el Tribunal: primera Dra. MARÍA TERESA DAY, segundo Dr. MARIO DANIEL ADARO, tercero Dr. OMAR ALEJANDRO PALERMO; cuarto Dr. PEDRO JORGE LLORENTE; quinto Dr. JOSÉ V. VALERIO; sexta Dra. MARINA ISUANI, y séptima Dra. SILVINA MIQUEL.\n\nA

In [95]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(data)

Created a chunk of size 1286, which is longer than the specified 1000
Created a chunk of size 1566, which is longer than the specified 1000
Created a chunk of size 2088, which is longer than the specified 1000
Created a chunk of size 1032, which is longer than the specified 1000
Created a chunk of size 1111, which is longer than the specified 1000
Created a chunk of size 1577, which is longer than the specified 1000
Created a chunk of size 1149, which is longer than the specified 1000
Created a chunk of size 1690, which is longer than the specified 1000
Created a chunk of size 1359, which is longer than the specified 1000
Created a chunk of size 1174, which is longer than the specified 1000
Created a chunk of size 1731, which is longer than the specified 1000
Created a chunk of size 1747, which is longer than the specified 1000
Created a chunk of size 1139, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of s

In [ ]:
texts[20]

In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings)


In [ ]:
retriever = db.as_retriever()
retrieved_docs = retriever.invoke(
    "Cual es la primera cuestion?"
)
print(retrieved_docs[0].page_content)

In [48]:
index = VectorstoreIndexCreator(embedding=embeddings,).from_loaders([loader])

In [ ]:
index.vectorstore.delete_collection()

## Ask LLM to summarize each section

In [19]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Antecedentes' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
#summary_ant = (index.query(query,llm=llm))
summary_ant = llm(f"\n\nHuman: {query} {data[0].page_content}\n\nAssistant")

print(summary_ant)

 Aquí está un resumen de mi análisis del fallo:

- Se trata de un fallo de la Suprema Corte de Justicia de Mendoza en convocatoria plenaria para determinar la constitucionalidad del artículo 3 de la Ley Provincial 9.017. 

- Dicho artículo establece un plazo de 45 días hábiles judiciales para formalizar recursos ante el fuero laboral provincial contra resoluciones de Comisiones Médicas Jurisdiccionales en materia de riesgos del trabajo, bajo apercibimiento de caducidad.

- Por mayoría, la Corte resolvió que el artículo es constitucional. 

- Los jueces Day, Llorente, Valerio y Miquel sostuvieron la constitucionalidad del artículo, argumentando que no se vulnera el control judicial suficiente ni el acceso a la justicia, ya que se prevé una acción ordinaria laboral de amplio debate. También señalaron que la regulación de plazos procesales es facultad de las provincias.

- Los jueces Adaro, Palermo e Isuani votaron por la inconstitucionalidad, considerando que la norma avanza sobre facult

In [ ]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion ''Sobre la primera cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"

sub_docs = index.vectorstore.similarity_search(query)
print(sub_docs[0].page_content)

In [60]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Sobre la primera cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_1 = (index.query_with_sources(query,llm=llm))
print(summary_1)


{'question': "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Sobre la primera cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:", 'answer': ' La ley n° 9.017 en su art. 1 es constitucional y convencional al disponer la adhesión de la Provincia de Mendoza a las disposiciones contenidas en el Título I de la Ley Nacional nº 27.348, con las modificaciones y adecuaciones establecidas por la presente ley. La conformidad legislativa local se formalizó con la sanción de la Ley n° 9017 que dispuso la adhesión de la Provincia de Mendoza a las disposiciones contenidas en el Título I de la Ley Nacional N° 27.348.\n\n', 'sources': '/home/harpo/JurisGPT/rawdata/laboral/10000003368.pdf'}


In [61]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Sobre la segunda cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_2 = (index.query(query,llm=llm))
print(summary_2)



La Sección Segunda de nuestra Corte Suprema de Mendoza, en su sentencia del día 16 de junio de 2022, ha analizado y discutido la cuestión de si la normativa de fondo o de código puede ser aplicada a los accidentes del trabajo en la provincia de Mendoza. La Corte ha determinado que, en este caso específico, la normativa de fondo no es aplicable ya que la ley nº 9.017, que establece el procedimiento administrativo obligatorio para los accidentes del trabajo en la provincia de Mendoza, se encuentra en contradicción con el Código Civil y la Constitución Nacional. La Corte ha destacado que la normativa de fondo debe ser aplicada de manera uniforme en todo el país, y no puede darse una interpretación diferente según el lugar donde ocurra el accidente del trabajo. Por lo tanto, la normativa de código debe aplicarse en este caso, y no la normativa de fondo.


In [62]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Sobre la tercera cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_3 = (index.query(query,llm=llm))
print(summary_3)



La seccción "Sobre la tercera cuestión" de la sentencia discute la conformidad legislativa local y su relación con la ley n° 9.017, que dispuso la adhesión de la Provincia de Mendoza a las disposiciones contenidas en el Título I de la Ley Nacional n° 27.348, y su adecuación constitucional por la Suprema Corte de Justicia de Mendoza. La sección también menciona la delegación de competencias a la jurisdicción administrativa nacional y el plazo de apelación establecido por la ley n° 9.024. En resumen, la seccción analiza la conformidad legislativa local y su relación con la normatividad nacional en materia de riesgos del trabajo.


In [63]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resumen de la seccion 'Resuelve' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_res = (index.query(query,llm=llm))
print(summary_res)



The Resuelve section of this court sentence in Mendoza, Argentina, addresses the issue of whether the province of Mendoza has the authority to delegate its powers to the national administrative jurisdiction regarding labor accidents. The court held that the province's adhesion to the national law on labor accidents does not imply a complete delegation of its powers, but rather a recognition of the national jurisdiction's competence in this matter. The court also noted that the province's autonomy is respected in matters that are not delegable. This decision was based on the principle of uniform codification and the need to avoid confusion and disparities in the application of labor laws across different provinces.


In [64]:
prompt = f"Eres un miembro de la corte suprena de Mendoza en Argentina. A partir del siguiente resumen de una setencia de la corte :\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"
prompt +="Explicar las causas detras de la decision final de la sentencia. Escribe un sumario de la corte en no menos de 250 palabras. Favor de no mencionar:\n1. Los nombres de las secciones.\n2. Los nombres de los casos.\n3. Los nombres de los ministros de la corte.\n4. Las fechas y la ubicacion de la corte.\n\n"
prompt +="Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia." 
print(prompt)

Eres un miembro de la corte suprena de Mendoza en Argentina. A partir del siguiente resumen de una setencia de la corte :

```


En este sentido, la presente sección "Antecedentes" se refiere a la adhesión de la Provincia de Mendoza a las disposiciones contenidas en el Título I de la Ley Nacional nº 27.348, que establece el procedimiento administrativo obligatorio para los accidentes del trabajo. La adhesión fue formalizada mediante la sanción de la Ley n° 9017 que delegó expresamente a la jurisdicción administrativa nacional las competencias necesarias para dar cumplimiento al procedimiento administrativo obligatorio establecido por la norma precitada. Sin embargo, existen pronunciamientos de sus integrantes en favor de la inconstitucionalidad del precepto en la misma causa "Manrique". Además, los planteos recurrentes y las decisiones disímiles que también se han adoptado en las distintas Cámaras del Trabajo de la Provincia de Mendoza, muestran a las claras la necesidad del presente p

In [20]:
prompt = f"\n\nHuman:Eres un miembro de la corte suprena de Mendoza en Argentina. A partir del siguiente resumen de una setencia de la corte :\n\n```\n {data[0].page_content} \n```\n\n"
prompt +="Explicar las causas detras de la decision final de la sentencia. Escribe un sumario de la corte en no menos de 250 palabras. Favor de no mencionar:\n1. Los nombres de las secciones.\n2. Los nombres de los casos.\n3. Los nombres de los ministros de la corte.\n4. Las fechas y la ubicacion de la corte.\n\n"
prompt +="Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia.\n\nAssistant:" 
summary = llm(prompt)

In [21]:
print(summary)

 Basado en el resumen de la sentencia, la Corte Suprema de Justicia de Mendoza se pronunció sobre la constitucionalidad del artículo 3 de la Ley Provincial 9017, que establece un plazo de caducidad de 45 días hábiles para interponer una acción judicial ordinaria contra las resoluciones de las Comisiones Médicas Jurisdiccionales en materia de riesgos del trabajo. 

Por mayoría, la Corte consideró que dicho artículo es constitucional. Para llegar a esta conclusión, analizó en profundidad el marco normativo de la Ley 9017, su relación con la Ley Nacional 27348 a la que adhiere la provincia, y los antecedentes jurisprudenciales sobre la materia tanto a nivel local como nacional.

La Corte sostuvo que la instancia administrativa previa ante las Comisiones Médicas es válida, y que la provincia tiene facultades para regular cuestiones procesales como el plazo para accionar judicialmente. Destacó que la acción ordinaria prevista en la Ley 9017 permite un control judicial amplio y es más benefi

# FUNCTIONS

## Create summary (ES)

In [26]:
def create_summary_es(rule_file, llm):
    #loader = UnstructuredPDFLoader(rule_file)
    loader = TextLoader(rule_file)
    data = loader.load()


    prompt = f"\n\nHuman: Eres un oficial perito de la corte suprema de Mendoza en Argentina. A partir del siguiente fallo de una sentencia de la corte :\n\n```\n{data[0].page_content}.\n```\n\n"    
    prompt+= "escribir un resumen de cada una de las secciones.\n\n Assistant: "

   # prompt = f"\n\nHuman: Eres un oficial perito de la corte suprema de Mendoza en Argentina. A partir del siguiente resumen de una sentencia de la corte :\n\n```\n{data[0].page_content}.\n```\n\n"    
   # prompt +="Armar un documento sumario en donde se expliquen las causas detras de la decision final de la sentencia. El sumario de la corte debe contener al menos 4096 palabras."
   # prompt +="Es importante determinar cuál es la doctrina del fallo (lo que resulta de un sumario). Por otra parte, no se deben mezclar los votos de la sentencia cuando se resuelve por mayoría de votos confirmar la sentencia de Cámara y el voto del ministro preopinante queda en minoría." 
   # prompt +="No utilizar las frases: 'La Sentencia resuelve', o 'La Corte Confirmó', 'La Corte resolvió', 'La sentencia se basa en', 'La Corte estableció'"
    
   # prompt +="Favor de no mencionar NUNCA:\n1. Los nombres de las secciones como ser: primera cuestion, segunda cuestion o tercera cuestion.\n2. Los nombres de los casos.\n3. Los nombres de los ministros de la corte.\n4. Las fechas y la ubicacion de la corte.\n5. La palabra corte\n\n"
   # prompt +="Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia. En caso de disidencia indicar el nombre del ministro de la corte.\n\nAssistant: " 
   # print(prompt)
   
    print("calculating prompt")
    #print(prompt)
    res = llm(prompt)
    return res

In [19]:
def create_summary_es_old(rule_file, llm):
    loader = UnstructuredPDFLoader(rule_file)
    data = loader.load()
    
    prompt = f"\n\nHuman: Eres un oficial perito de la corte suprema de Mendoza en Argentina. A partir del siguiente resumen de una sentencia de la corte :\n\n```\n{data[0].page_content}.\n```\n\n"    
    prompt +="Presentar un sumario en donde se expliquen las causas detras de la decision final de la sentencia. Escribir el sumario de la corte en no mas de 700 palabras."
    prompt +="Favor de no mencionar NUNCA:\n1. Los nombres de las secciones como ser: primera cuestion, segunda cuestion o tercera cuestion.\n2. Los nombres de los casos.\n3. Los nombres de los ministros de la corte.\n4. Las fechas y la ubicacion de la corte.\n5. La palabra corte\n\n"
    prompt +="Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia. En caso de disidencia indicar el nombre del  ministro de la corte.\n\nAssistant:" 
    #print(prompt)
   
    print("calculating prompt")
    #print(prompt)
    res = llm(prompt)
    return res

In [14]:
file_name = "/home/harpo/JurisGPT/rawdata/laboral/sumariosbigdb/fallos/10000000790.txt"
summary = create_summary_es(file_name, llm = llm)

calculating prompt


In [12]:
print(summary)

 Aquí está mi resumen de la sentencia de la Corte Suprema de Justicia de Mendoza en 1200 palabras:

El actor René Jorge Argañaraz sufrió un accidente laboral mientras trabajaba para Jesús Carlos Fantelli e Hijos. Sufrió un desgarro en los gemelos de la pierna derecha al levantar un tacho pesado. La ART Responsabilidad Patronal le brindó algunas prestaciones pero luego se negó a cubrir el accidente. Argañaraz demandó a Responsabilidad Patronal por $72.037 en concepto de indemnización por incapacidad laboral. 

Tras la liquidación de Responsabilidad Patronal, se hizo parte en el juicio Prevención ART SA, designada por la SSN para hacerse cargo de las prestaciones adeudadas. La Cámara del Trabajo declaró la inconstitucionalidad del artículo 14.2.b de la LRT y condenó a Prevención ART a pagar $109.906 más intereses.

Contra esa sentencia, Argañaraz y Prevención ART interpusieron recursos extraordinarios de inconstitucionalidad y casación. 

El recurso de Argañaraz se declaró parcialmente p

In [67]:
output_directory_path = "/root/JurisGPT/data/laboral/qa/"  
name, extension = os.path.splitext(os.path.basename(file_name))
summary_filename = f"{name}_summary.txt"
summary_file_path = os.path.join(output_directory_path, summary_filename)
with open(summary_file_path, 'w') as f:
    f.write(summary)


## Create summary from dir

In [29]:
def create_summary_from_dir(directory_path, output_directory_path, llm):
    import time
    # Loop through each file in the directory
    for filename in os.listdir(directory_path):
        # Extract the file extension
        name, extension = os.path.splitext(filename)
        
        # Build the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Skip directories, only work on PDF files
        #if os.path.isfile(file_path) and extension.lower() == '.pdf':
        
        # Skip directories, only work on TXT files
        if os.path.isfile(file_path) and extension.lower() == '.txt':
            # Save summary to new file
            summary_filename = f"{name}_summary_claude.txt"
            summary_file_path = os.path.join(output_directory_path, summary_filename)
            if not os.path.isfile(summary_file_path):
                print(f'working on {file_path}')
                # Generate summary
                start_time = time.time()
                summary = create_summary_es(file_path, llm)
                stop_time = time.time()
        
                with open(summary_file_path, 'w') as f:
                    f.write(summary)
                print(f'created {summary_file_path} in {stop_time - start_time} seconds')
            else:
                print(f'{summary_file_path} already created, skipping...')

## Calculate ROUGE metric

In [34]:
import os
import re
from rouge import Rouge

def calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern):
    # Initialize Rouge
    rouge = Rouge()
    
    # Compile the regular expressions for filename patterns
    system_re = re.compile(system_filename_pattern)
    model_re = re.compile(model_filename_pattern)
    
    # Initialize storage for scores
    all_scores = []

    # Loop through the files in the system summaries directory
    for system_filename in os.listdir(system_dir):
        match = system_re.match(system_filename)
        if not match:
            continue

        # Extract ID
        system_id = match.groups()[0]

        # Find the corresponding model summary file using ID
        model_filename = f"{system_id}_summary.txt"
        model_path = os.path.join(model_dir, model_filename)

        # Check if model summary exists
        if not os.path.exists(model_path):
            print(f"Model summary for {system_id} not found. Attempted to open {model_path}")
            continue

        

        # Check if model summary exists
        if not os.path.exists(model_path):
            print(f"Model summary for {system_id} not found.")
            continue

        # Read system and model summaries
        with open(os.path.join(system_dir, system_filename), 'r', encoding='utf-8') as f:
            system_summary = f.read().strip()
            system_summary = re.sub('<.*?>', '', system_summary)
            #print(system_summary)
        
        with open(model_path, 'r', encoding='utf-8') as f:
            model_summary = f.read().strip()
    
        # Compute the ROUGE score
        score = rouge.get_scores(system_summary, model_summary)[0]
        #all_scores.append(score)
        all_scores.append({
            'system_id': system_id,
            'rouge-1': score['rouge-1'],
            'rouge-2': score['rouge-2'],
            'rouge-l': score.get('rouge-l', {})  # Add this line to include ROUGE-L
        })
        # Compute the ROUGE score


        # Extract ROUGE-1, ROUGE-2, and ROUGE-3 scores
        #rouge_1 = score.get('rouge-1', {})
        #rouge_2 = score.get('rouge-2', {})
       
        # Pretty-print scores
        #print(f"ROUGE scores for ID {system_id}:")
        #print(f"  ROUGE-1: R: {rouge_1.get('r', 'N/A'):.4f}, P: {rouge_1.get('p', 'N/A'):.4f}, F: {rouge_1.get('f', 'N/A'):.4f}")
        #print(f"  ROUGE-2: R: {rouge_2.get('r', 'N/A'):.4f}, P: {rouge_2.get('p', 'N/A'):.4f}, F: {rouge_2.get('f', 'N/A'):.4f}")
      
        
        #print(f"ROUGE score for ID {system_id}: {score}")

    # If you want average scores, you can calculate that here from all_scores.
    # ...

    return all_scores


## Print ROUGE TABLE

In [35]:
import pandas as pd  # Don't forget to import pandas

def print_rouge_table(all_scores):
    pd.set_option('display.max_rows', None)
    
    system_ids = []
    metrics = []
    rs = []
    ps = []
    fs = []

    # Loop through each dictionary to extract the values
    for score_dict in all_scores:
        system_id = score_dict['system_id']

        for metric, values in score_dict.items():
            if metric == 'system_id':
                continue  # Skip the system_id, we've already saved it

            system_ids.append(system_id)
            metrics.append(metric)
            rs.append(values['r'])
            ps.append(values['p'])
            fs.append(values['f'])

    # Create the DataFrame
    df_long = pd.DataFrame({
        'system_id': system_ids,
        'metric': metrics,
        'r': rs,
        'p': ps,
        'f': fs
    })

    return df_long

# MAIN

## Create Summaries

In [31]:
directory_path = "/home/harpo/JurisGPT/rawdata/laboral/sumariosbigdb/fallos/"  
output_directory_path = "/home/harpo/JurisGPT/rawdata/laboral/sumariosbigdb/resumenfallos/"  
create_summary_from_dir(directory_path, output_directory_path, llm = llm)

/home/harpo/JurisGPT/rawdata/laboral/sumariosbigdb/resumenfallos/10000003177_summary_claude.txt already created, skipping...
/home/harpo/JurisGPT/rawdata/laboral/sumariosbigdb/resumenfallos/10000001477_summary_claude.txt already created, skipping...
/home/harpo/JurisGPT/rawdata/laboral/sumariosbigdb/resumenfallos/10000001897_summary_claude.txt already created, skipping...
/home/harpo/JurisGPT/rawdata/laboral/sumariosbigdb/resumenfallos/10000002498_summary_claude.txt already created, skipping...
/home/harpo/JurisGPT/rawdata/laboral/sumariosbigdb/resumenfallos/10000003419_summary_claude.txt already created, skipping...
/home/harpo/JurisGPT/rawdata/laboral/sumariosbigdb/resumenfallos/10000003626_summary_claude.txt already created, skipping...
/home/harpo/JurisGPT/rawdata/laboral/sumariosbigdb/resumenfallos/10000002387_summary_claude.txt already created, skipping...
/home/harpo/JurisGPT/rawdata/laboral/sumariosbigdb/resumenfallos/10000002696_summary_claude.txt already created, skipping...


## Calculate ROUGE

The ROUGE-1 metric is a type of Recall-Oriented Understudy for Gisting Evaluation (ROUGE) score that is commonly used for evaluating the quality of summaries by comparing them to reference summaries. Specifically, ROUGE-1 measures the overlap of unigrams (single words) between the generated summary and the reference summary.

The ROUGE-1 score comprises three values:

1. **Precision (P)**: This measures the number of overlapping unigrams found in both the generated summary and the reference summary, divided by the total number of unigrams in the generated summary. In essence, precision answers the question: "Of all the unigrams in the generated summary, how many are in the reference summary?"
   \[
   \text{Precision} = \frac{\text{Number of overlapping unigrams}}{\text{Total number of unigrams in generated summary}}
   \]

2. **Recall (R)**: This measures the number of overlapping unigrams found in both the generated summary and the reference summary, divided by the total number of unigrams in the reference summary. Recall answers the question: "Of all the unigrams in the reference summary, how many are in the generated summary?"
   \[
   \text{Recall} = \frac{\text{Number of overlapping unigrams}}{\text{Total number of unigrams in reference summary}}
   \]

3. **F1 Score (F)**: This is the harmonic mean of precision and recall and serves as a single number that balances the trade-off between precision and recall.
   \[
   \text{F1 Score} = \frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
   \]

ROUGE-1 is particularly useful for assessing the quality of extractive summaries, where sentences or phrases are selected from the original text to form a summary. The metric can also be used for abstractive summaries but may not capture the quality of the summary as effectively as when combined with other ROUGE metrics like ROUGE-2 (bigram overlap) or ROUGE-L (longest common subsequence).

In [37]:
# Paths and patterns
system_dir = '/home/harpo/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/home/harpo/JurisGPT/data/laboral/qa/claude_test/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [38]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

,system_id,metric,r,p,f
0,10000003247,rouge-1,0.230769,0.235294,0.233010
3,10000003300,rouge-1,0.391566,0.268595,0.318627
6,10000003275,rouge-1,0.320652,0.241803,0.275701
9,10000003220,rouge-1,0.253623,0.271318,0.262172
12,10000003292,rouge-1,0.476510,0.221184,0.302128
15,10000003219,rouge-1,0.291971,0.117647,0.167715
18,10000003307,rouge-1,0.276923,0.165644,0.207294
21,10000003368,rouge-1,0.372881,0.224490,0.280255
24,10000003258,rouge-1,0.307018,0.134615,0.187166
27,10000003221,rouge-1,0.267176,0.180412,0.215385


In [39]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

(0.3189089277416128, 0.07481111247211031)

# 7B Results

In [112]:
# Paths and patterns
system_dir = '/root/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/root/JurisGPT/data/laboral/qa/7b/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [114]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

,system_id,metric,r,p,f
0,10000003275,rouge-1,0.245455,0.221311,0.232759
3,10000003307,rouge-1,0.319277,0.162577,0.215447
6,10000003220,rouge-1,0.179856,0.193798,0.186567
9,10000003219,rouge-1,0.276730,0.129412,0.176353
12,10000003258,rouge-1,0.327381,0.211538,0.257009
15,10000003300,rouge-1,0.322404,0.243802,0.277647
18,10000003368,rouge-1,0.227027,0.214286,0.220472
21,10000003221,rouge-1,0.349315,0.262887,0.300000
24,10000003292,rouge-1,0.229665,0.149533,0.181132
27,10000003247,rouge-1,0.262195,0.210784,0.233696


In [115]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

(0.27393049419520055, 0.05468252084765027)

# 13B results

In [44]:
# Paths and patterns
system_dir = '/home/harpo/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/home/harpo/JurisGPT/data/laboral/qa/13b/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [45]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

,system_id,metric,r,p,f
0,10000003247,rouge-1,0.286765,0.191176,0.229412
3,10000003300,rouge-1,0.379032,0.194215,0.256831
6,10000003275,rouge-1,0.283688,0.163934,0.207792
9,10000003220,rouge-1,0.141935,0.170543,0.154930
12,10000003292,rouge-1,0.299320,0.137072,0.188034
15,10000003219,rouge-1,0.314050,0.111765,0.164859
18,10000003307,rouge-1,0.352459,0.131902,0.191964
21,10000003368,rouge-1,0.230216,0.163265,0.191045
24,10000003258,rouge-1,0.281879,0.161538,0.205379
27,10000003221,rouge-1,0.315789,0.247423,0.277457


In [46]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

(0.28851332177949185, 0.06561640326263223)

# 70B results

In [41]:
# Paths and patterns
system_dir = '/home/harpo/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/home/harpo/JurisGPT/data/laboral/qa/70b/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [42]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

,system_id,metric,r,p,f
0,10000003247,rouge-1,0.292857,0.200980,0.238372
3,10000003300,rouge-1,0.391892,0.239669,0.297436
6,10000003275,rouge-1,0.289773,0.209016,0.242857
9,10000003220,rouge-1,0.123529,0.162791,0.140468
12,10000003292,rouge-1,0.285714,0.143302,0.190871
15,10000003219,rouge-1,0.267176,0.102941,0.148620
18,10000003307,rouge-1,0.251366,0.141104,0.180747
21,10000003368,rouge-1,0.297521,0.183673,0.227129
24,10000003258,rouge-1,0.347222,0.192308,0.247525
27,10000003221,rouge-1,0.366013,0.288660,0.322767


In [43]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

(0.29130631075130886, 0.07395673921972185)